In [35]:
import numpy             as np
import gvar              as gv
import matplotlib.pyplot as plt
import lsqfit

In [23]:
from b2heavy import FnalHISQMetadata

In [24]:
from b2heavy.TwoPointFunctions.utils     import correlation_diagnostics
from b2heavy.TwoPointFunctions.types2pts import CorrelatorIO, plot_effective_coeffs
from b2heavy.TwoPointFunctions.fitter    import StagFitter

In [25]:
from b2heavy.ThreePointFunctions.types3pts  import Ratio, RatioIO
from b2heavy.ThreePointFunctions.fitter3pts import RatioFitter

Select a case for which we want to calculate the recoil parameter

In [26]:
ENSEMBLE = 'Coarse-1'
DATA_DIR = '/Users/pietro/code/data_analysis/BtoD/Alex/'

BINSIZE  = {
    'MediumCoarse': 13,
    'Coarse-1': 11
} 
SMSLIST  = ['1S-1S','d-d','d-1S']

COV_SPECS = dict(
    diag   = False,
    block  = False,
    scale  = True,
    shrink = True,
    cutsvd = 0.01
)

In [27]:
mdata = FnalHISQMetadata.params(ENSEMBLE)
LVOL   = mdata['L']
alphas = mdata['alphaS'] 

## Two-pts functions

In [28]:
MESON      = 'Dst'
MOM_LIST   = ['000','100','200','300','110','211']
TRANGE_EFF = {
    'Coarse-1': {
        '000': (13,20),
        '100': (13,20),
        '200': (13,20),
        '300': (10,17),
        '110': (11,19),
        '211': (11,19)
    }
}
TRANGE     = {
    'Coarse-1': {
        '000': (5,26),
        '100': (5,23),
        '200': (5,21),
        '300': (5,17),
        '110': (5,23),
        '211': (5,19)
    }
}

Perform fit at each mom and collect fundamental energies

In [ ]:
energies = {}
for mom in MOM_LIST:
    io   = CorrelatorIO(ENSEMBLE,MESON,mom,PathToDataDir=DATA_DIR)
    stag = StagFitter(
        io       = io,
        jkBin    = BINSIZE[ENSEMBLE],
        smearing = SMSLIST
    )

    effm,effa = stag.meff(
        TRANGE_EFF[ENSEMBLE][mom],
        **COV_SPECS
    )

    fit = stag.fit(
        Nstates = 3,
        trange  = TRANGE[ENSEMBLE][mom],
        **COV_SPECS
    )

    energies[mom] = fit.p['E']

E = {mom: energies[mom][0] for mom in energies}

Define functions for dispersion relation

In [34]:
def mom_to_p2(mom,L=2*np.pi):
    return sum([(2*np.pi/L*float(px))**2 for px in mom])

def dispersion_relation_lsqfit(pveclist,d):
    M1,M2,M4,w4 = d['M1'],d['M2'],d['M4'],d['w4']

    res = []
    for pvec in pveclist:
        p2  = sum(pvec**2)
        p22 = p2**2
        p4  = sum(pvec**4)

        res.append(
            M1**2 + (M1/M2 * p2) + ((1/M1**2 - M1/M4**3)/4 * p22) - (M1*w4/3 * p4)
        )

    return np.array(res)


Fit the dispersion relation

In [39]:
psort = list(E.keys())
psort.sort(key=lambda x: mom_to_p2(x,L=LVOL))

pv = [2*np.pi/LVOL*np.array([float(px) for px in mom]) for mom in psort]
p2 = [sum(np.array([float(px)*2*np.pi/LVOL for px in mom])**2) for mom in psort]
E0 = np.asarray([E[kp] for kp in psort])    

priors = dict(
    M1 = gv.gvar(0.5,1.5),
    M2 = gv.gvar(0.5,1.5),
    M4 = gv.gvar(0.5,1.5),
    w4 = gv.gvar(0.5,1.5)
)
fit = lsqfit.nonlinear_fit(
    data  = (pv,E0**2),
    fcn   = dispersion_relation_lsqfit,
    prior = priors
)

print(fit)


Least Square Fit:
  chi2/dof [dof] = 0.074 [6]    Q = 1    logGBF = 6.3723

Parameters:
             M1   1.0796 (51)     [  0.5 (1.5) ]  
             M2     1.07 (24)     [  0.5 (1.5) ]  
             M4     0.66 (21)     [  0.5 (1.5) ]  
             w4     0.3 (1.4)     [  0.5 (1.5) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 8/0.0)
  fitter = scipy_least_squares    method = trf



1.0796(51)

In [40]:
M1,M2 = fit.p['M1'], fit.p['M2']

## Three-pts functions

In [107]:
RATIO  = 'xfstpar'
MOM    = '300'
SMLIST = ['1S']

Nstates = 1
trange  = (3,11)

Initialize ratio objects

In [108]:
io = RatioIO(ENSEMBLE,RATIO,mom,PathToDataDir=DATA_DIR)
ratio = RatioFitter(
    io,
    jkBin    = BINSIZE[ENSEMBLE],
    smearing = SMLIST
)


Set priors with $\Delta E$

In [109]:
x,y = ratio.format()

Kpr    = gv.gvar(y['1S'][x.tolist().index(ratio.Ta//2)].mean,0.05)
dE_src = gv.gvar(
    energies[mom][2].mean,
    energies[mom][2].sdev*3
)

Perform the fit

In [110]:
fit = ratio.fit(
    Nstates = Nstates,
    trange  = trange,
    priors  = ratio.priors(Nstates,K=Kpr),
    **COV_SPECS
)

pr = fit.prior
popt = dict(fit.pmean)
fcov = gv.evalcov(fit.y)
c2,ce,p = ratio.chi2exp(Nstates,trange,popt,fcov)

Least Square Fit:
  chi2/dof [dof] = 0.4 [9]    Q = 0.94    logGBF = 35.437

Parameters:
        ratio 0   -0.086 (25)     [ -0.074 (50) ]  
       dE_src 0    -1.38 (85)     [  -1.5 (1.0) ]  
         A_1S 0    -0.57 (20)     [     0 ± 1.0 ]  
         B_1S 0    -0.23 (22)     [     0 ± 1.0 ]  

Settings:
  svdcut/n = 0/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 16/0.0)
  fitter = scipy_least_squares    method = trf



In [111]:
XF = fit.p['ratio'][0]

## Confront

In [112]:
w1 = np.sqrt(1+mom_to_p2(MOM,L=LVOL)/M1**2)
w2 = np.sqrt(1+mom_to_p2(MOM,L=LVOL)/M2**2)
wr = (1+XF**2)/(1-XF**2)

In [113]:
print(f'{w1 = }, {w2 = }, {wr = }')

w1 = 1.1392(12), w2 = 1.141(59), wr = 1.0148(88)
